# Qualifier

+ Qualifica um outfit, atribuindo-lhe um score.

+ **Input**: Embedding das peças
+ **Output**: Score do outfit

Desenvolvida uma CNN básica com multi-input e single output. O objetivo desta rede é o de obter algo para conseguir criar o serviço do sugestor. Maior desenvolvimento é necessário para obter resultados utilizáveis.

A rede foi treinada com o dataset de outfits do Polyvore.

**Dataset**: https://drive.google.com/file/d/0B4Eo9mft9jwoNm5WR3ltVkJWX0k/view?usp=sharing

**Features**: nomes das imagens correspondentes a outfits. Os nomes são construidos da seguinte maneira "outfit"_"artigo"

**Labels**: 0 -> Caso o conjunto de artigos não componha um outfit
        1 -> Caso contrário
        
Os pesos da rede desenvolvida estão armazenados em prototype_cnn.h5

In [ ]:
import pandas as pd
import numpy as np

dataset_path = "../../datasets/Polyvore"

In [ ]:
fashion_df = pd.read_csv(dataset_path+"/fashion_compatibility_prediction.txt",
                             sep=' ',
                             header=None,
                              names=['label','item_1','item_2','item_3','item_4','item_5','item_6','item_7','item_8'],
                            keep_default_na=False)

In [ ]:
fashion_df.head()

In [ ]:
print("Size:", fashion_df.shape[0])

In [ ]:
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def getImagePath(image_name):
    if image_name != '':
        name = image_name.split("_")
        return dataset_path + "/images/"+name[0]+"/"+name[1]+".jpg"
    else:
        return ''

In [ ]:
def loadImage(image_path):
    #print(image_path)
    if image_path != '':
        bgr_img = cv2.imread(image_path)
        bgr_img = cv2.resize(bgr_img, (400,400))
    else:
        bgr_img = np.zeros((400,400, 3), dtype="uint8")
    return bgr_img

In [ ]:
def showOutfit(outfit):
    count = 0
    items = list()
    for item in outfit:
        if item != '':
            count += 1
            item_path = getImagePath(item)
            items.append(item_path)
    fig = plt.figure(figsize=(20,20))
    for i in range(count):
        ax = fig.add_subplot(1,count, i+1, xticks=[], yticks=[])
        image = loadImage(items[i])
        print('Image size:', image.shape)
        ax.imshow(image)
        ax.set_title('item %s' % (i+1))

In [ ]:
print("Image:",fashion_df.iloc[1,2])
print("at:", getImagePath(fashion_df.iloc[1,2]))

In [ ]:
showOutfit(fashion_df.iloc[1, 1:5])

In [ ]:
from sklearn.model_selection import train_test_split

fashion_df = fashion_df[0:500]

X_train, X_test, y_train, y_test = train_test_split( 
    fashion_df.iloc[:, 1 : 5], 
    fashion_df.iloc[:, 0], test_size=0.20,
    random_state=42)

X_train = X_train[0:500]
y_train = y_train[0:500]

In [ ]:
X_train_images = X_train.applymap(getImagePath)
X_train_images = X_train_images.applymap(loadImage)
X_train_images = np.array(X_train_images)
X_train_images = X_train_images / 255.0

X_test_images = X_test.applymap(getImagePath)
X_test_images = X_test_images.applymap(loadImage)
X_test_images = np.array(X_test_images)
X_test_images = X_test_images / 255.0

In [ ]:
def unwrapArray(array, new_shape):
    unwrapped_array = np.zeros(new_shape)
    for i in range(array.shape[0]):
        for j in range(array[0].shape[0]):
            unwrapped_array[i,j,:,:,:] = array[i,j]
    return unwrapped_array

In [ ]:
x_train_array = unwrapArray(X_train_images, (X_train_images.shape[0],X_train_images.shape[1],64,64,3))
x_test_array = unwrapArray(X_test_images, (X_test_images.shape[0],X_test_images.shape[1],64,64,3))

In [ ]:
x_train_array.shape

In [ ]:
y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
x_train_array = x_train_array.transpose(1,0,2,3,4)
x_test_array = x_test_array.transpose(1,0,2,3,4)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import concatenate
np.random.seed(1000)

In [ ]:
def build_cnn():
    image = Input(shape=(64,64,3))
    
    chanDim = -1
    
    x = Conv2D(16, (3, 3), padding="same")(image)
    x = Activation("relu")(x)
    x = BatchNormalization(axis=chanDim)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    
    x = Conv2D(32, (3, 3), padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization(axis=chanDim)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    
    x = Flatten()(x)
    x = Dense(16)(x)
    x = Activation("relu")(x)
    x = BatchNormalization(axis=chanDim)(x)
    x = Dropout(0.5)(x)
    
    model = Model(inputs = image, outputs = x)
    return model

In [ ]:
model_1 = build_cnn()
model_2 = build_cnn()
model_3 = build_cnn()
model_4 = build_cnn()

combinedInput = concatenate([model_1.output, model_2.output, model_3.output, model_4.output])

x = Dense(128, activation="relu")(combinedInput)
x = Dense(64, activation="relu")(x)
x = Dense(32, activation="relu")(x)
x = Dense(1, activation="sigmoid")(x)

model = Model(inputs=[model_1.input, model_2.input, model_3.input, model_4.input], outputs=x)

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit(
    [x_train_array[0], x_train_array[1], x_train_array[2], x_train_array[3]], y_train,
    epochs=10, validation_split=0.1)

In [ ]:
loss, accuracy = model.evaluate([x_test_array[0],x_test_array[1],x_test_array[2],x_test_array[3]], y_test, verbose = False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

In [ ]:
model.save_weights('prototype_cnn.h5')